##  melon 주간 가요 top100차트를 크롤링한다. 
### ("국내종합"), 2016이전 "종합", 수정, 2016이전 "가요"
- year단위로 수행한다.
- 2017 이후
- https://www.melon.com/chart/search/list.htm?chartType=WE&classCd=DM0000&startDay=20161226&endDay=20170101&moved=Y
- 2016 이전
- https://www.melon.com/chart/search/list.htm?chartType=WE&classCd=DP0000&startDay=20160307&endDay=20160313&moved=Y
- 2008
- https://www.melon.com/chart/search/list.htm?chartType=WE&classCd=CL0000&startDay=20080928&endDay=20081004&moved=Y
- ~2008 "가요" 장르로 수정
- https://www.melon.com/chart/search/list.htm?chartType=WE&classCd=CL0100&startDay=20080928&endDay=20081004&moved=Y

In [13]:
import requests
import os
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

### set yearly start_day

In [14]:
# 연도별 startday의 범위를 설정한다.
yearly_start = {
                "2005": ["20050103", "20051226"],
                "2006": ["20060102", "20061225"],
                "2007": ["20070101", "20071223"],
                "2008": ["20071230", "20081221"],
                "2009": ["20081228", "20091227"],    
                "2010": ["20100103", "20101226"],
                "2011": ["20110102", "20111225"],
                "2012": ["20120101", "20121224"],
                "2013": ["20121231", "20131223"],
                "2014": ["20131230", "20141222"],
                "2015": ["20141229", "20151228"],
                "2016": ["20160104", "20161226"],
                "2017": ["20170102", "20171225"],
                "2018": ["20180101", "20181224"],
                "2019": ["20181231", "20191223"],
                "2020": ["20191230", "20201228"],
                "2021": ["20210104", "20211227"],
                "2022": ["20220103", "20230101"],
                "2023": ["20230102", "20231231"]}

In [21]:
def getStartEndPairsByTargetYear(target_year):
    # 해당 연도의 startDay, endDay pairs를 만든다.
    #
    # 해당 연도 start_days의 범위를 꺼낸다.
    target_start1, target_start2 = yearly_start.get(target_year)
    # datetime obj로 변환.
    target_start1, target_start2 = datetime.strptime(target_start1, "%Y%m%d"), datetime.strptime(target_start2, "%Y%m%d")
    #
    # start_days, end_days를 채운다.
    start_days = []
    end_days = []
    while target_start1 <= target_start2:
        start_days.append(datetime.strftime(target_start1, "%Y%m%d"))
        # 2007년의 기준 변경 대응
        if target_start1 == datetime(year=2007, month=7, day=9):
            end_days.append(datetime.strftime(target_start1 + timedelta(days=5), "%Y%m%d"))
        else:
            end_days.append(datetime.strftime(target_start1 + timedelta(days=6), "%Y%m%d"))
        #
        target_start1 = target_start1 + timedelta(days=7)
        # 2012년 기준 변경의 대응
        if target_start1 == datetime(year=2012, month=8, day=12):
            target_start1 = datetime(year=2012, month=8, day=13)
        if target_start1 == datetime(year=2007, month=7, day=16):
            target_start1 = datetime(year=2007, month=7, day=15)
    #
    if target_start1 - timedelta(days=7) != target_start2:
        print("last start_day not matched")
        print(target_start1, target_start2)
    return start_days, end_days
    
    

In [22]:
target_year = "2023"
start_days, end_days = getStartEndPairsByTargetYear(target_year)

print("year: {}, {} weeks".format(target_year, len(start_days)))

last start_day not matched
2024-01-01 00:00:00 2023-12-31 00:00:00
year: 2023, 52 weeks


## crawl weekly & save Html

In [23]:
def requestsHtml(url):
    #
    PROXY_IP = "proxypool.daumsoft.com:7002"
    proxies = {"http": PROXY_IP, "https": PROXY_IP}
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
    headers = {"User-Agent": user_agent, "Host": "www.melon.com",
              "Cookie": "__T_=1; PCID=16847374038867242922976; PC_PCID=16847374038867242922976; POC=MP10; wcs_bt=s_f9c4bde066b:1684737405; _T_ANO=G9eN8NNSWyJ2AiGsAFhyuxlSlyUwodg1wdA2xQPBSr222rt9gLwHr4KPkodclmDLtZd0uRv5xET7IOb1f1+/tmXHI8WXQCWxJ5GahNlvcNblBJIM+b8/ogzo/e2c/+5trXBExVBUXrpRbj7tkwvZ8CUifzoSUoExK4vuqnMwLq94GR0x58Kc/5OTDeVRh9x0xmpBTYM5OYTbNoTdT3DFxZe4OZF/26Fm5bUmDfJA+KIWMgb/FgJ2ZZJ6HTziV2wSb7x6s9mGQsDu2r3KiHpthygxHNua4HB+7RBa0ytw+QC3mYUsK0K4aiu6uQLtBftNMO6eT92+qn9PkFpv+AMsgg==",
              }
    #resp = requests.get(url, headers=headers, proxies=proxies)
    resp = requests.get(url, headers=headers) #, proxies=proxies)
    #
    return resp

In [24]:
def makeMelonWeeklyChartUrl(start_day, end_day):
    #
    if start_day[:4] >= "2017":
        url_tmplt = "https://www.melon.com/chart/search/list.htm?chartType=WE&classCd=DM0000&startDay={start_day}&endDay={end_day}&moved=Y"
    elif start_day >= "20081026":
        # DP0100: 가요
        url_tmplt = "https://www.melon.com/chart/search/list.htm?chartType=WE&classCd=DP0100&startDay={start_day}&endDay={end_day}&moved=Y"
    else:
        # CL0100: 가요
        url_tmplt = "https://www.melon.com/chart/search/list.htm?chartType=WE&classCd=CL0100&startDay={start_day}&endDay={end_day}&moved=Y"
        
    #
    return url_tmplt.format(start_day=start_day, end_day=end_day)
    

In [25]:
def saveHtml(resp, target_year, file_path_name):
    #
    if target_year not in os.listdir("htmls"):
        os.mkdir("htmls/{}".format(target_year))
    #
    with open(file_path_name, "w", encoding='UTF-8') as of:
        print(resp.text, file=of)
    #
    return None

In [26]:
for cnt, (start_day, end_day) in enumerate(zip(start_days, end_days)):
    #
    url = makeMelonWeeklyChartUrl(start_day, end_day)
    #
    # check if already crawled
    file_path_name = "htmls/{}/{}-{}.html".format(target_year, start_day, end_day)
    if os.path.isfile(file_path_name):
        print("already crawled: {}-{}".format(start_day, end_day))
        continue
        
    print(url)
    
    #
    resp = requestsHtml(url)
    if not resp.text:
        print("empty html")
        break
    #
    saveHtml(resp, target_year, file_path_name)
    time.sleep(1)

already crawled: 20230102-20230108
already crawled: 20230109-20230115
already crawled: 20230116-20230122
already crawled: 20230123-20230129
already crawled: 20230130-20230205
already crawled: 20230206-20230212
already crawled: 20230213-20230219
already crawled: 20230220-20230226
already crawled: 20230227-20230305
already crawled: 20230306-20230312
already crawled: 20230313-20230319
already crawled: 20230320-20230326
already crawled: 20230327-20230402
already crawled: 20230403-20230409
already crawled: 20230410-20230416
already crawled: 20230417-20230423
already crawled: 20230424-20230430
already crawled: 20230501-20230507
already crawled: 20230508-20230514
already crawled: 20230515-20230521
https://www.melon.com/chart/search/list.htm?chartType=WE&classCd=DM0000&startDay=20230522&endDay=20230528&moved=Y
https://www.melon.com/chart/search/list.htm?chartType=WE&classCd=DM0000&startDay=20230529&endDay=20230604&moved=Y
https://www.melon.com/chart/search/list.htm?chartType=WE&classCd=DM0000&